In [22]:
import pandas as pd
import numpy as np
import json
import os

In [23]:
# Misc:
web_base = 'https://data.sandiego.gov/datasets/?department=police'

#### What we want to keep as a general outline for data

stop_id

stop_cause / reason_for_stop

result_of_stop

stopdate

stoptime

service_area

block

subject_race / race --> map to race codes

beat (new) --> map to service area

actions_taken (new)

officer_assignment_key (new)

exp_years (new)

stopduration (new)

perceived_age (new)

In [24]:
def get_table(yr):
    if yr < 2018:
        url = 'http://seshat.datasd.org/pd/vehicle_stops_{}_datasd_v1.csv'.format(yr)
        return pd.read_csv(url)
    
    else: # post- 2018
        # cols we care about
        cols_2018 = ['stop_id', 'date_stop', 'time_stop', 'stopduration', 
                 'officer_assignment_key', 'exp_years', 'beat', 
                 'perceived_age', 'gend']

        url = 'http://seshat.datasd.org/pd/ripa_stops_datasd_v1.csv'
        df = pd.read_csv(url)
        return df[cols_2018]

### columns we still need:

stop_cause / reason_for_stop

result_of_stop

actions_taken 

In [25]:
# other tables for POST- 2018
def get_merge_data():
    reason = 'stop_reason'
    reason_cols = ['stop_id', 'reason_for_stop', 'reason_for_stopcode']

    result = 'stop_result'
    result_cols= ['stop_id', 'result']

    race = 'race'
    race_cols = ['stop_id', 'race']

    action = 'actions_taken'
    action_cols = ['stop_id', 'action']

    base = 'http://seshat.datasd.org/pd/ripa_{}_datasd.csv'

    reason_df = pd.read_csv(base.format(reason))
    result_df = pd.read_csv(base.format(result))
    race_df = pd.read_csv(base.format(race))
    action_df = pd.read_csv(base.format(action))
    
    return [[reason_cols, result_cols, race_cols, action_cols], [reason_df, result_df, race_df, action_df]]

In [26]:
# get all the new cols from other urls
def gen_cols(df1, gen_df, cols):
    new = df1.merge(gen_df, on = 'stop_id')
    drop = [x for x in new.columns if x not in cols]
    new = new.drop(columns = drop).drop_duplicates(subset = 'stop_id')
    return new

In [27]:
def merge_data(tbl):
    merge_cols = get_merge_data[0]
    merge_dfs = get_merge_data[1]
    merged = []

    for i in range(len(merge_dfs)):
        merged.append(gen_cols(tbl, merge_dfs[i], merge_cols[i]))
          
    first = merged[0]
    for i in range(0, len(merged)):
        if i == len(merged) - 1:
            break 
        first = pd.merge(first, merged[i + 1], on = 'stop_id')

    return pd.merge(basic, first, on='stop_id').drop_duplicates(subset = 'stop_id')

### Clean column names

* These are the same: 'stop_id', 'date_stop', 'time_stop', 'service_area', 'subject_race'/'Race Code', 'subject_sex'/'gend',


* These are excess for post- 2018: 'beat', 'officer_assignment_key', 'exp_years', 'perceived_age'


* These are excess for pre- 2018: 
    - 'sd_resident'
    - 'arrested'
    - 'searched'
    - 'obtained_consent'
    - 'contraband_found'
    - 'property_seized'


&rightarrow;  Therefore: Need to make 'subject_race' == 'Race Code' and  'subject_sex' == 'gend'


* Additionally for pre- 2018, need to engineer: 'outcome'

    -  from ['arrested', 'searched', 'property_seized'] in pre- 2018 and ['action', 'result'] in post- 2018


* Columns we are ok with having lots of null values because they are useful for analysis:

    - 'sd_resident'
    - 'beat'
    - 'officer_assignment_key'
    - 'exp_years'
    - 'perceived_age'

In [28]:
# only for PRE- 2018
def change_bool(string):
        if (string == 'Y') | (string =='y'):
            return 1
        if (string == 'N') | (string == 'n'):
            return 0
        return np.nan

def map_bool(cols, df):
    for col in cols:
        if col not in list(df.columns):
            continue
        df[col] = df[col].apply(lambda x: c_bool(x))
    return df

# only for POST- 2018
def change_sex(x):
    if (x == 1.0) | (x == 1):
        return 'M'
    elif (x == 2.0) | (x == 2):
        return 'F'
    else:
        return np.nan

def map_sex(col):
    return col.apply(lambda x: change_sex(x))

In [29]:
# accounts for both formats
def clean_bool(df, yr):
    if yr < 2018:
        c = ['sd_resident', 'searched', 'contraband_found', 'property_seized', 'arrested']
        return map_bool(c, df)
    
    else:
        df['gend'] = map_sex(df['gend'])
        return df

### columns we still need to map:

subject_race / race - map to race codes

beat - map to service area

In [30]:
# only for POST- 2018
def map_race(df):
    # engineered from existing file
    # race_codes = 'http://seshat.datasd.org/pd/vehicle_stops_race_codes.csv'
    # and intution

    race_dict = {
     'Asian' : 'A',
     'OTHER ASIAN': 'A',
     'Middle Eastern or South Asian': 'M',
     'BLACK': 'B',
     'CHINESE': 'C',
     'CAMBODIAN': 'D',
     'FILIPINO': 'F',
     'GUAMANIAN': 'G',
     'HISPANIC': 'H',
     'Hispanic/Latino/a': 'H',
     'INDIAN': 'I',
     'JAPANESE': 'J',
     'KOREAN': 'K',
     'LAOTIAN': 'L',
     'OTHER': 'O',
     'PACIFIC ISLANDER': 'P',
     'Pacific Islander': 'P',
     'SAMOAN': 'S',
     'HAWAIIAN': 'U',
     'VIETNAMESE': 'V',
     'WHITE': 'W',
     'White': 'W',
     'ASIAN INDIAN': 'Z',
     'Native American': 'N'
    }
    df['Race Code'] = df['race'].map(race_dict)
    return df

# only for POST- 2018
def map_service_area(df):
    stop_beats = 'http://seshat.datasd.org/sde/pd/pd_beats_datasd.geojson'
    beats = gpd.read_file(stop_beats)
    # get unique beats
    unique_beats = beats[['beat', 'serv']].drop_duplicates('beat')
    beat_dict = dict(zip(unique_beats.beat, uni.serv))
    df['service_area'] = ['beat'].map(beat_dict)
    return df

In [31]:
def rename_cols(df):
    return df.rename(columns={'Race Code': 'subject_race', 'gend': 'subject_sex', 'reason_for_stop': 'stop_cause'})

In [32]:
# Outcome possible values: 

# 'None'
# 'Warning (verbal or written)' 
# 'Search of property was conducted' 
# 'Property was seized' >> property seized
# 'Custodial Arrest without warrant'

def check_outcome(row, year):
    if year < 2018:
        # check 'arrested', 'searched', 'property_seized'
        if row.arrested == 1.0:
            return 'Arrest'
        elif row.property_seized == 1.0:
            return 'Property was seized'
        elif row.searched == 1.0:
            return 'Search of property was conducted'
        return 'Not Applicable'
        
    else:
        a = row.action
        r = row.result
        
        # check worst outcome first
        if r == 'Custodial Arrest without warrant':
            return r
        elif (a == 'Search of property was conducted') | (a == 'Property was seized'):
            return a
        elif r == 'Warning (verbal or written)':
            return r
        
        return 'Not Applicable'
    
def outcome_map(df, yr):
    df_copy = df.copy()
    df_copy['Outcome'] = df_copy.apply(lambda x: check_outcome(x, yr), axis=1)
    
    if yr < 2018:
        return df_copy.drop(columns=['arrested', 'property_seized', 'searched', 'obtained_consent', 'contraband_found'])
        
    return df_copy.drop(columns=['action', 'result'])

In [33]:
def pre_2018_format(df, yr):
    df = clean_bool(df, yr)
    df = outcome_map(df, yr)
    return df

def post_2018_format(df, yr):
    df = clean_bool(df, yr)
    df = map_race(df)
    df = map_service_area(df)
    df = rename_cols(df)
    df = outcome_map(df, yr)
    return df

In [34]:
def format_df(df, yr):
    order = ['stop_id', 'stop_cause', 'reason_for_stopcode', 'date_stop', 'time_stop', 'stopduration', 'stop_cause', 
         'Outcome', 'beat', 'service_area', 'subject_race', 'subject_sex', 'perceived_age', 'sd_resident', 
         'officer_assignment_key', 'exp_years']
    
    cols = [i for i in order if i in list(df.columns)]
    if yr < 2018:
        return pre_2018_format(df, yr)[cols]
    
    else:
        return post_2018_format(df, yr)[cols]

In [35]:
def get_data(yr):
    tbl = get_table(yr)
    
    if yr < 2018:
        return format_df(tbl, yr)

    else:
        df = merge_data(tbl)
        return format_df(df, yr)

In [36]:
get_data(2018)

KeyboardInterrupt: 

# Part 1

#### Cleaning
Perform an initial EDA to statistically assess the quality of the data and its appropriateness for addressing the problem at hand, justifying data cleaning logic. This will likely address issues with accuracy, precision, and missingness of specific attributes, tying these issues to their possible impact over eventual results.

#### Descriptive Stats
Statistically summarize the relevant, cleaned attributes and derived features (e.g. in univariate and bivariate analyses) for San Diego.

#### Traffic Stop Analysis
Calculate and document the differences in stop rates and post-stop outcomes. The analysis should address possible reasons for such differences (including addressing possible confounders). Additionally: - The significance of these differences should be tested using statistical inference. - These differences should also be calculated across other variables of interest (e.g. service area).

#### Veil of Darkness
Perform the Veil of Darkness analysis for San Diego. Include an introduction to the technique and interpret the results. (In Assignment #3 you will carefully address the shortcoming of this result.)

# Part 2

Develop code to clean data (as defined and justified in Part 1), create the features for the replication, and compute the statistics for the report. Such code should conform to the methodology portion of the course (e.g. using the project template).

In particular, your project should have a run.py with the following targets:

* data creates the data needed for analysis.
* process cleans and prepares the data for analysis (e.g. cleaning and feature creation).
* data-test ingests a small amount of test data (that process can then process).